In [37]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [13]:
url = 'https://www.indeed.com/jobs?q=data%20analyst%20entry%20level&l=remote&start=0&vjk=ff725c70949a666a'
req = requests.get(url)
bsoup = bs(req.content, 'html.parser')
pages = []
page_num = bsoup.find('div', {'id': 'searchCountPages'}).text.strip()
for num in page_num.split():
    if num.isdigit():
        pages.append(num)
page = int(pages[1])
page

72

In [31]:
def get_html(page):
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36'}
    url = f'https://www.indeed.com/jobs?q=data%20analyst%20entry%20level&l=remote&start={page}&vjk=ff725c70949a666a'
    r = requests.get(url, header)
    soup = bs(r.content, 'html.parser')
    return soup
    

In [40]:
def get_data(soup):
    
    divs = soup.find_all('div', class_='job_seen_beacon')
    for item in divs:
        title = item.find('h2').text.strip()
        company = item.find('span', class_='companyName').text.strip()
        try:
            salary = item.find('div', class_='attribute_snippet').text.strip()
        except:
            salary = ''
        summary = item.find('div', class_='job-snippet').text.strip().replace('\n', ' ')
        job = {'title': title, 'company': company, 'salary': salary, 'summary': summary}
        jobs.append(job)
    return
jobs = []
for i in range(0, page, 10):
    soup = get_html(0)
    get_data(soup)
    
len(jobs)

120

In [41]:
data = pd.DataFrame(jobs)

In [42]:
data.head()

,title,company,salary,summary
0,Business/Data Analyst (Entry Level),Agama Solutions,$30 an hour,Role: Business/Data Analyst (Entry Level). 1+ ...
1,Data Analyst I,"Payspan, Inc",Full-time,"The Data Analyst I, is an entry level position..."
2,Jr. Business Analyst,Kajeet,Full-time,They need to be able to pull relevant data fro...
3,Entry Level Business Analyst,Agama Solutions,$30 - $35 an hour,Troubleshoot data issues by validating differe...
4,Data Specialist,YipitData,,Be the primary owner running or building new d...


In [44]:
data.to_csv('indeed_jobs.csv')